In [1]:
%load_ext dotenv
%dotenv

# Tool calling

In this notebook we explore how to do tool calling in practice.  
We will use various libraries to see how they implement tool calling.

## OpenAI

Let us start with OpenAI's own library.

In [2]:
from openai import OpenAI

client = OpenAI()

Let's give the model a tool to work with.  
Tools can be anything, but since we are working with python, it makes sense to use a python function.

In [3]:
def get_weather(location: str):
    """
    Get current temperature for a given location.
    """
    return f"It temperature is 20 degrees Celsius in {location}."

The model won't understand what a python function is though,  
so we have to present it to the model in a way that it can understand.  
Since this is an OpenAI model, we need to follow the OpenAI tool calling format.

In [4]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

We can now invoke the model as follows:

In [5]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)
print(completion.to_json(indent=2))

{
  "id": "chatcmpl-BHrPObmqQDFNcKNerfr03t9x9CIYF",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "tool_calls": [
          {
            "id": "call_OvP9hHBg1D4I5npiGMvNtmVE",
            "function": {
              "arguments": "{\"location\":\"Paris, France\"}",
              "name": "get_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1743596578,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_b376dfbbd5",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 65,
    "total_tokens": 82,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    

We got a bit output including some metadata.  
Let's look at the message:

In [6]:
print(completion.choices[0].message.to_json(indent=2))

{
  "content": null,
  "refusal": null,
  "role": "assistant",
  "annotations": [],
  "tool_calls": [
    {
      "id": "call_OvP9hHBg1D4I5npiGMvNtmVE",
      "function": {
        "arguments": "{\"location\":\"Paris, France\"}",
        "name": "get_weather"
      },
      "type": "function"
    }
  ]
}


The model expressed that it wants to make a tool call `get_weather(location="Paris, France")`.  
Now it is up to use to invoke the python function.

In [7]:
import json
kwargs_string = completion.choices[0].message.tool_calls[0].function.arguments
kwargs = json.loads(kwargs_string)
tool_result = get_weather(**kwargs)
tool_result

'It temperature is 20 degrees Celsius in Paris, France.'

You can see that we had to take a couple of steps.  
The model outputs a string, which **should** be a valid json string.  
We have to parse the json string into a python dictionary.  
Then we have the arguments we need to pass to the function.  
Finally we can call the function.

We can now return the tool result to the model and let it continue.

In [8]:
second_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is the weather like in Paris today?"},
        completion.choices[0].message,
        {
        "role": "tool",
        "tool_call_id": completion.choices[0].message.tool_calls[0].id,
        "content": tool_result
        }
    ],
    tools=tools,
)
print(second_completion.choices[0].message.to_json(indent=2))

{
  "content": "The weather in Paris today is 20 degrees Celsius.",
  "refusal": null,
  "role": "assistant",
  "annotations": []
}


Great, we allowed the model to make a tool call and it finally responded to the user prompt.

That was quite a lot of work though.  
This is a common pattern and it makes sense to abstract it away.  

Indeed, OpenAI realised this themselves and introduced the Agents SDK.  
This recently released API is a convenient wrapper around the tool calling process.  
We can now define a tool from a python function as follows.

In [9]:
from agents import function_tool

@function_tool  
def get_weather(location: str):
    """
    Get current temperature for a given location.
    """
    return f"It temperature is 20 degrees Celsius in {location}."

Agents SDK will handle the conversion of the function into the expected format.  
Note that the docstring will also be passed to the model,  
but the model will see nothing from the body of the function.

In [10]:
print("Name:", get_weather.name)
print("Description:", get_weather.description)
print("Parameters:")
print(json.dumps(get_weather.params_json_schema, indent=2))

Name: get_weather
Description: Get current temperature for a given location.
Parameters:
{
  "properties": {
    "location": {
      "title": "Location",
      "type": "string"
    }
  },
  "required": [
    "location"
  ],
  "title": "get_weather_args",
  "type": "object",
  "additionalProperties": false
}


Now we can define our agent and run it.

In [11]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    model="gpt-4o-mini",
    tools=[get_weather],
)

res = await Runner.run(agent, "What is the weather like in Paris today?")
res.final_output

'The temperature in Paris today is 20 degrees Celsius. If you need more specific weather details, feel free to ask!'

Wow, that was a lot easier!  
The Agents SDK took care of:  

- the conversion of the python method into the expected format.
- The parsing of model response.
- The invocation of the tool python method.
- Looping on the previous steps until the model is done.

Note that the model is "done" when it responds with a regular message instead of a tool call.


The Agents SDK is a nice framework for creating agents, but there are many others.  
For example, we have been using LangChain and LangGraph for this purpose for a while.  
Let's remind ourselves of how to do the same in LangGraph.  
First, we create the tool.

In [12]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def get_weather(location: Annotated[str, "The location to get the weather for."]):
    """
    Get current temperature for a given location.
    """
    return f"It temperature is 20 degrees Celsius in {location}."

We can similarly get the schema of the tool.

In [13]:
print(json.dumps(get_weather.tool_call_schema.model_json_schema(), indent=2))

{
  "description": "Get current temperature for a given location.",
  "properties": {
    "location": {
      "description": "The location to get the weather for.",
      "title": "Location",
      "type": "string"
    }
  },
  "required": [
    "location"
  ],
  "title": "get_weather",
  "type": "object"
}


Note that
```
def get_weather(location: Annotated[str, "The location to get the weather for."]):
```
allowed us to annotate the `location` argument with a description.  
This will also be passed to the model and can be used to help the model understand what the function does.

Now let's create and run the agent.

In [14]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model=ChatOpenAI(model="gpt-4o-mini")
react_agent = create_react_agent(
    name="Assistant",
    model=model,
    tools=[get_weather],
)

state = react_agent.invoke({"messages": [("user", "What is the weather like in Paris today?")]})
state['messages'][-1].pretty_print()

================================== Ai Message ==================================
Name: Assistant

The weather in Paris today is 20 degrees Celsius.


Nice! The api is pretty similar to the OpenAI Agents SDK.  
But we do not get a vendor lock-in effect, because we can simply switch to another LLM provider.

In [15]:
import os
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama

model = ChatOllama(model="mistral-small", base_url=os.getenv("OLLAMA_BASE_URL"))
react_agent = create_react_agent(
    name="Assistant",
    model=model,
    tools=[get_weather],
)

state = react_agent.invoke({"messages": [("user", "What is the weather like in Paris today?")]})
state['messages'][-1].pretty_print()

================================== Ai Message ==================================
Name: Assistant

It is 20 degrees Celsius in Paris.
